# Notebook testing out visualisation of the moon as a 3d sphere

Mostly taken from the tutorial here:
https://towardsdatascience.com/create-interactive-globe-earthquake-plot-in-python-b0b52b646f27

In [1]:
import plotly

In [6]:
import xarray as xa
import rioxarray as rio

In [24]:
data.x.size

4096

In [168]:
def get_data(fname = 'lroc_color_poles_4k.tif',
             ds_factor = 2):
    """ 
    Gets the data from a geotif with filename fname, downscaling int the processs
    Inputs:
        fname (str) : filename of geotif to load
        ds_factor (int) downscale factor
    Output:
        lon, lat, data ([n_lat x n_lon] np arrays)
    """
    import rioxarray as rio
    import xarray as xa
    import numpy as np
    from rasterio.enums import Resampling
    
    xds = rio.open_rasterio(fname)
    xds.attrs['crs'] = 'EPSG:4326'
    downscale_factor = 2
    new_width = xds.rio.width // ds_factor
    new_height = xds.rio.height // ds_factor
    xds.var

    xds_resampled = xds.rio.reproject(
        xds.rio.crs,
        shape=(new_height, new_width),
        resampling=Resampling.bilinear,
        SRC_METHOD='NO_GEOTRANSFORM'
    )
    lon = xds_resampled.x.data
    lat = xds_resampled.y.data
    
    lon = (lon - lon.min()) / (lon.max() - lon.min()) * 360 - 180 
    lat = (lat - lat.min()) / (lat.max() - lat.min()) * 180 - 90 


    
    
    lon, lat = np.meshgrid(lon, lat)
    
    return lon, lat, xds_resampled.data[0, ...]

In [171]:
import numpy as np
def degree2radians(degree):
  # convert degrees to radians
  return degree*np.pi/180
  
def mapping_map_to_sphere(lon, lat, radius=1):
    # this function maps the points of coords (lon, lat) to points onto the sphere of radius radius
    lon=np.array(lon, dtype=np.float64)
    lat=np.array(lat, dtype=np.float64)
    lon=degree2radians(lon)
    lat=degree2radians(lat)
    xs=radius*np.cos(lon)*np.cos(lat)
    ys=radius*np.sin(lon)*np.cos(lat)
    zs=radius*np.sin(lat)
    return xs, ys, zs


In [204]:
# Get mesh-shape topography data
lon, lat, greys = get_data(ds_factor = 5)

In [205]:


# Locations on the sphere fro grid cells in x,y,z coords
xs, ys, zs = mapping_map_to_sphere(lon, lat)

In [206]:
np.max(lon)

180.0

In [222]:
Ctopo = [[0, 'rgb(0, 0, 0)'],[1.0, 'rgb(255,255,255)']]

In [227]:
cmin = 0
cmax = 255
topo_sphere=dict(type='surface',
  x=xs,
  y=ys,
  z=zs,
  colorscale=Ctopo,
  surfacecolor=greys,
  cmin=cmin,
  cmax=cmax)

In [263]:
import plotly.graph_objs as go

titlecolor = 'white'
bgcolor = 'black'




noaxis=dict(showbackground=False,
  showgrid=False,
  showline=False,
  showticklabels=False,
  ticks='',
  title='',
  zeroline=False)

layout = go.Layout(
  autosize=False, width=1200, height=800,
  title = '3D Moonquake Map',
  titlefont = dict(family='Courier New', color=titlecolor),
  showlegend = False,
  scene = dict(
    xaxis = noaxis,
    yaxis = noaxis,
    zaxis = noaxis,
    aspectmode='manual',
    aspectratio=go.layout.scene.Aspectratio(
      x=1, y=1, z=1)),
  paper_bgcolor = bgcolor,
  plot_bgcolor = bgcolor)

In [264]:
from plotly.offline import plot

plot_data=[topo_sphere]
fig = go.Figure(data=plot_data, layout=layout)
plot(fig, validate = False, filename='index.html',
   auto_open=True)

'index.html'

# Loading moonquake data

In [300]:
!pwd

/home/leigh/Documents/Projects/moonquake


In [359]:
import pandas as pd
import glob

###############----For input----###############
file_list = glob.glob("cleaned_data.csv")
data = pd.read_csv(file_list[i])

# Data selection   

# Change format to datetime for event date
data['np_DateTime']=pd.to_datetime(data['Date'],format='%Y-%m-%d %H:%M:%S')

date = np.array(data['np_DateTime'])
evlon = np.array(data['Long'])
evlat = np.array(data['Lat'])

# create color col
moonquake_labels = ['Shallow Moonquake', 'Deep Moonquake']
data['type_num'] = pd.Series([int(d in moonquake_labels) for d in data['Type']])
# # Calculate time difference
# RefTime = np.datetime64("2000-01-01T00:00:00.000000000")
# Timefrom_RefTime = (date - RefTime)
# # For years
# Timefrom_RefYears = Timefrom_RefTime.astype('timedelta64[Y]')
# Timefrom_RefYears = Timefrom_RefYears / np.timedelta64(1, 'Y')

In [360]:
pd.Series([('_dm_' in d) for d in data['Source']])

0      False
1      False
2      False
3      False
4      False
       ...  
253     True
254     True
255     True
256     True
257     True
Length: 258, dtype: bool

In [361]:
data

,Unnamed: 0,Lat,Long,Date,RawType,Type,Source,np_DateTime,type_num
0,0,-3.94,-21.20,1969-11-20 22:17:00,12 LM,Artifical Impacts,gagnepian_2006_catalog,1969-11-20 22:17:00,0
1,1,-2.75,-27.86,1970-04-15 02:09:00,13 S-IVB,Artifical Impacts,gagnepian_2006_catalog,1970-04-15 02:09:00,0
2,2,-8.09,-26.02,1971-02-04 07:40:00,14 S-IVB,Artifical Impacts,gagnepian_2006_catalog,1971-02-04 07:40:00,0
3,3,-3.42,-19.67,1971-02-07 00:45:00,14 LM,Artifical Impacts,gagnepian_2006_catalog,1971-02-07 00:45:00,0
4,4,-1.51,-11.81,1971-07-29 20:58:00,15 S-IVB,Artifical Impacts,gagnepian_2006_catalog,1971-07-29 20:58:00,0
...,...,...,...,...,...,...,...,...,...
253,253,7.10,22.50,1977-09-13 03:58:00,dm,Deep Moonquake,nakamura_2005_dm_locations,1977-09-13 03:58:00,1
254,254,42.90,110.90,1977-09-13 09:55:00,dm,Deep Moonquake,nakamura_2005_dm_locations,1977-09-13 09:55:00,1
255,255,54.20,56.50,1977-05-12 09:44:00,dm,Deep Moonquake,nakamura_2005_dm_locations,1977-05-12 09:44:00,1
256,256,24.80,35.70,1977-02-12 21:38:00,dm,Deep Moonquake,nakamura_2005_dm_locations,1977-02-12 21:38:00,1


In [362]:
xs_ev_org, ys_ev_org, zs_ev_org = mapping_map_to_sphere(evlon, evlat)

#Create color bar in Matplotlib
import matplotlib
from matplotlib import cm

def matplotlib_to_plotly(cmap, pl_entries):
    h = 1.0/(pl_entries-1)
    pl_colorscale = []
    
    for k in range(pl_entries):
        C = list(map(np.uint8, np.array(cmap(k*h)[:3])*255))
        pl_colorscale.append([k*h, 'rgb'+str((C[0], C[1], C[2]))])
        
    return pl_colorscale

def MlibCscale_to_Plotly(cbar):
    cmap = matplotlib.cm.get_cmap(cbar)
    rgb = []
    norm = matplotlib.colors.Normalize(vmin=0, vmax=255)

    for i in range(0, 255):
        k = matplotlib.colors.colorConverter.to_rgb(cmap(norm(i)))
        rgb.append(k)

    Cscale = matplotlib_to_plotly(cmap, 255)
    
    return Cscale

cbar = 'jet_r'
Cscale_EQ = MlibCscale_to_Plotly(cbar)

colours = data['type_num'].to_numpy()

In [363]:
ratio = 1. 
xs_ev = xs_ev_org*ratio
ys_ev = ys_ev_org*ratio
zs_ev = zs_ev_org*ratio

ratio = 1.01
xs_ev_up = xs_ev_org*ratio
ys_ev_up = ys_ev_org*ratio
zs_ev_up = zs_ev_org*ratio

In [364]:
depmax = 1.
depmin = 0.
depbin = 1.

cmin = depmin
cmax = depmax
cbin = depbin

seis_3D_depth_up = go.Scatter3d(x = xs_ev_up,
                      y = ys_ev_up,
                      z = zs_ev_up,
                      mode='markers',
                      name='measured',
                      marker = dict(
                          size = 3.,
                          cmax = cmax,
                          cmin = cmin,
                          colorbar = dict(
                              title = 'Source Depth',
                              titleside = 'right',
                              titlefont = dict(size = 16, 
                                               color = titlecolor, 
                                               family='Courier New'),
                              tickmode = 'array',
                              ticks = 'outside',
                              ticktext = list(np.arange(cmin,cmax+cbin,cbin)),
                              tickvals = list(np.arange(cmin,cmax+cbin,cbin)),
                              tickcolor = titlecolor,
                              tickfont = dict(size=14, color = titlecolor,
                                             family='Courier New')
                          ),
                          ### choose color option
                          color = colours,
                          ### choose color option
                          colorscale = Cscale_EQ,
                          showscale = True,
                          opacity=1.),
                      hoverinfo='skip'
                      )

In [365]:
from plotly.offline import plot

plot_data=[topo_sphere, seis_3D_depth_up]
fig = go.Figure(data=plot_data, layout=layout)
plot(fig, validate = False, filename='index2.html',
   auto_open=True)

'index2.html'